In [5]:
import os, json
from datetime import datetime
from dotenv import load_dotenv
from google import genai
from google.genai import types  # <- para GenerateContentConfig

# =========================
# Config
# =========================
load_dotenv()

# En el SDK nuevo no hace falta el prefijo "models/"
# Si lo traés así desde el entorno, se lo quitamos para evitar 404.
_MODEL_ENV = os.getenv("GOOGLE_GEMINI_MODEL", "gemini-2.5-flash")
MODEL = _MODEL_ENV.replace("models/", "")
# El cliente toma GEMINI_API_KEY o GOOGLE_API_KEY desde el entorno
client = genai.Client()

# === Config única (editar solo acá) ===
CONFIG = {
    "from_mailreader": True,
    "limit": 200, #limite de mails a buscar
    "days_back": 30, #cantidad de dias hacia atras para buscar mails
    "json_path": None,
    "max_chars": 120_000,
    "out": None,                 # "respuesta.txt" o None
    "model": MODEL,              # o "gemini-2.5-pro", etc.
    "temperature": None,         # ej. 0.3
    "max_output_tokens": None,   # ej. 1200
    "response_mime_type": None,  # ej. "text/markdown"
    "debug": False,
}


# =========================
# Helpers
# =========================

def load_rows_from_json(path: str) -> list[dict]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if isinstance(data, dict) and "rows" in data:
        return data["rows"]
    if not isinstance(data, list):
        raise ValueError("El JSON no es una lista de correos.")
    return data

def load_rows_from_mailreader(limit: int | None = None, days_back: int | None = None) -> list[dict]:
    """
    Importa mailreader13 y obtiene los correos en memoria.
    Cambiá 'mailreader13' si tu módulo se llama diferente.
    """
    try:
        import mailreader13_beta as mailreader
    except Exception as e:
        raise RuntimeError(f"No pude importar mailreader13: {e}")

    if hasattr(mailreader, "collect_rows"):
        # Pasamos limit/days_back si tu collect_rows los soporta
        try:
            rows = mailreader.collect_rows(limit=limit, days_back=days_back or None, debug=False)
        except TypeError:
            # fallback por si tu firma solo acepta limit
            rows = mailreader.collect_rows(limit=limit)
    elif hasattr(mailreader, "main"):
        out_path = os.getenv("OUTPUT_JSON", "emails.json")
        mailreader.main()
        rows = load_rows_from_json(out_path)
    else:
        raise RuntimeError("mailreader no expone collect_rows() ni main().")

    return rows

def clamp_rows(rows: list[dict], max_chars: int = 120_000) -> list[dict]:
    """Devuelve filas *compactas* y recorta por caracteres de forma consistente."""
    compact_rows, total = [], 0
    for r in rows:
        compact = {
            "Fecha": r.get("Fecha"),
            "ReceivedUTC": r.get("ReceivedUTC"),
            "Remitente": r.get("Remitente"),
            "RemitenteNombre": r.get("RemitenteNombre"),
            "Asunto": r.get("Asunto"),
            "Cuerpo": (r.get("Cuerpo") or "")[:2000],  # límite duro por mail
            "TieneAdjuntos": bool(r.get("TieneAdjuntos")),
            "Leido": bool(r.get("Leido")),
            "Respondido": bool(r.get("Respondido")),
            "Categorias": r.get("Categorias"),
            "WebLink": r.get("WebLink"),
        }
        s = json.dumps(compact, ensure_ascii=False)
        if total + len(s) > max_chars and compact_rows:
            break
        compact_rows.append(compact)
        total += len(s)
    return compact_rows

def build_prompt(rows: list[dict]) -> tuple[str, list[str]]:
    """
    Devuelve (system_instruction, contents_list) en el formato esperado por google-genai.
    - system_instruction: string con las reglas
    - contents_list: lista[str] con el payload (el SDK la convierte a UserContent)
    """
    system_instruction = (
        "Eres un asistente que analiza correos de Outlook. "
        "Responde en español, claro y estructurado, en menos de 200 palabras."
        "IMPORTANTE: Oculta datos sensibles reemplazando los nombres de personas por [REDACTED] y las direcciones de correo electronico por [MAIL]. "
        "No inventes ni modifiques información, solo redacta lo sensible. Entrega:\n"
        "1) Panorama (tendencias y temas) en 5-8 bullets.\n"
        "2) Top prioridades (máx 10) con acción sugerida.\n"
        "3) Fechas/calendario detectados (si se mencionan).\n"
        "4) Adjuntos relevantes (asunto).\n"
        "5) KPIs: total, sin leer, con adjuntos, respondidos.\n"
        f"Fecha de generación: {datetime.now().isoformat(timespec='seconds')}."
    )

    contents_list = [
        "A continuación va la lista JSON de correos.",
        json.dumps(rows, ensure_ascii=False),
    ]
    return system_instruction, contents_list


# =========================
# Ejecutor
# =========================
def run_analysis(config: dict):
    """Ejecuta el análisis leyendo TODO de config."""
    from_mailreader     = config["from_mailreader"]
    limit               = config["limit"]
    days_back           = config["days_back"]
    json_path           = config["json_path"]
    max_chars           = config["max_chars"]
    out                 = config["out"]
    model               = (config["model"] or MODEL).replace("models/", "")
    temperature         = config["temperature"]
    max_output_tokens   = config["max_output_tokens"]
    response_mime_type  = config["response_mime_type"]
    debug               = config["debug"]

    # 1) Carga de correos
    if from_mailreader:
        rows = load_rows_from_mailreader(limit=limit, days_back=days_back)
    else:
        jp = json_path or os.getenv("OUTPUT_JSON", "emails.json")
        rows = load_rows_from_json(jp)

    if not rows:
        return "", {"error": "No hay correos para analizar."}

    # 2) Orden + recorte
    rows_sorted  = sorted(rows, key=lambda r: r.get("ReceivedUTC") or r.get("Fecha") or "", reverse=True)
    rows_compact = clamp_rows(rows_sorted, max_chars=max_chars)

    # 3) Prompt
    system_instruction, contents = build_prompt(rows_compact)

    # 4) Llamada al modelo
    try:
        response = client.models.generate_content(
            model=model,
            contents=contents,
            config=types.GenerateContentConfig(
                system_instruction=system_instruction,
                max_output_tokens=max_output_tokens if max_output_tokens else None,
                temperature=temperature if temperature else None,
                response_mime_type=response_mime_type if response_mime_type else None,
            ),
        )
    except Exception as e:
        if debug:
            raise
        return "", {"error": f"Fallo generate_content: {e}", "model": model}

    text = getattr(response, "text", "") or ""

    # 5) Guardado opcional
    saved_path = None
    if out:
        saved_path = os.path.abspath(out)
        with open(out, "w", encoding="utf-8") as f:
            f.write(text)

    # 6) Métricas de uso
    usage = {}
    if hasattr(response, "usage_metadata") and response.usage_metadata:
        u = response.usage_metadata
        usage = {
            "prompt_tokens": getattr(u, "prompt_token_count", None),
            "response_tokens": getattr(u, "candidates_token_count", None),
            "thoughts_tokens": getattr(u, "thoughts_token_count", None),
            "total_tokens": getattr(u, "total_token_count", None),
        }

    meta = {
        "model": model,
        "saved_to": saved_path,
        "usage": usage,
        "rows_in": len(rows),
        "rows_after_clamp": len(rows_compact),
    }
    return text, meta


# =========================
# Invocación
# =========================
texto, meta = run_analysis(CONFIG)
print(texto)
meta

MSAL cache backend → Keychain (macOS) signal=/Users/jprizzi/.mailreader_msal_signal
Aquí tienes el análisis de tus correos:

**1) Panorama (Tendencias y Temas):**
*   Gestión financiera y contable (autónomos, proveedores, imputaciones, informes).
*   Coordinación de tareas contables y revisión de saldos con [REDACTED].
*   Novedades tecnológicas (GPT-5 en Microsoft 365 Copilot).
*   Asuntos legales y de RRHH (procedimiento por fallecimiento de empleado).
*   Solicitud y envío de documentación impositiva.
*   Gestión de inversiones (venta de bonos argentinos).
*   Comunicación y seguimiento de tareas pendientes.

**2) Top Prioridades con Acción Sugerida:**
1.  **AUTONOMOS 08-2025 (Vence 08-9-2025):** Verificar débito automático o usar VEP enviado por [REDACTED].
2.  **Proveedores/Contabilidad:** Coordinar con [REDACTED] la revisión de imputaciones y actualización de sistemas.
3.  **Fallecimiento Colaborador:** Revisar procedimiento legal y contable para el empleado fallecido.
4.  **Oper

{'model': 'gemini-2.5-flash',
 'saved_to': None,
 'usage': {'prompt_tokens': 4442,
  'response_tokens': 441,
  'thoughts_tokens': 1103,
  'total_tokens': 5986},
 'rows_in': 8,
 'rows_after_clamp': 8}